In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Define the regression models
def get_regression_models():
    return {
        "Linear Regression": LinearRegression(),
        "Ridge Regression": Ridge(),
        "Lasso Regression": Lasso(),
        "ElasticNet Regression": ElasticNet(),
        "Support Vector Regression (SVR)": SVR(),
        "Decision Tree Regression": DecisionTreeRegressor(),
        "Random Forest Regression": RandomForestRegressor(),
        "Gradient Boosting Regression": GradientBoostingRegressor(),
        "AdaBoost Regression": AdaBoostRegressor(),
        "XGBoost Regression": XGBRegressor()
    }

# Function to scale R^2 values using Min-Max Scaling and shift to avoid 0 or negative values
def scale_r2(r2_scores):
    scaler = MinMaxScaler(feature_range=(0.001, 1))  # Set the feature range to start from 0.001 to avoid 0
    r2_scaled = scaler.fit_transform(np.array(r2_scores).reshape(-1, 1))
    return r2_scaled.flatten()

# Function to preprocess data
def preprocess_data(dataset):
    # Replace NaN values with the column mean
    return dataset.fillna(dataset.mean())

# Function to train and evaluate models
def evaluate_models(X_train, X_test, y_train, y_test, models, dataset_name):
    results = []
    cv_results = {"RMSE": {}, "R^2": {}}
    kf = KFold(n_splits=10, shuffle=True, random_state=42)
    
    for name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)
        
        # Predict on train and test data
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)
        
        # Compute metrics for train and test
        rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
        rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
        r2_train = r2_score(y_train, y_train_pred)
        r2_test = r2_score(y_test, y_test_pred)
        
        # 10-fold cross-validation
        cv_rmse = -cross_val_score(model, X_train, y_train, cv=kf, scoring='neg_root_mean_squared_error')
        cv_r2 = cross_val_score(model, X_train, y_train, cv=kf, scoring='r2')
        
        # Scale R^2 values to avoid 0 or negative values
        cv_r2_scaled = scale_r2(cv_r2)
        r2_scaled = scale_r2([r2_test])[0]
        
        # Store results
        results.append([name, rmse_train, rmse_test, r2_train, r2_scaled, cv_rmse.mean(), cv_rmse.std(), cv_r2_scaled.mean(), cv_r2_scaled.std()])
        cv_results["RMSE"][name] = cv_rmse
        cv_results["R^2"][name] = cv_r2_scaled
    
    # Create a DataFrame for the results
    results_df = pd.DataFrame(results, columns=[
        "Model", "Train RMSE", "Test RMSE", "Train R^2", "Test R^2 (Scaled)", "CV RMSE Mean", "CV RMSE Std", "CV R^2 Mean (Scaled)", "CV R^2 Std (Scaled)"
    ])
    print(f"\nResults for {dataset_name}:\n", results_df)
    return results_df, cv_results

# Visualization functions
def plot_metrics(results_df, dataset_name):
    # Plot Test R^2 (Scaled) and Test RMSE for both Training and Testing datasets
    plt.figure(figsize=(14, 6))
    
    # Barplot for Test R^2 (Scaled)
    plt.subplot(1, 2, 1)
    sns.barplot(data=results_df, x="Model", y="Test R^2 (Scaled)", palette="viridis")
    plt.xticks(rotation=90)
    plt.title(f"Test R^2 (Scaled) for {dataset_name}")
    
    # Barplot for Test RMSE
    plt.subplot(1, 2, 2)
    sns.barplot(data=results_df, x="Model", y="Test RMSE", palette="magma")
    plt.xticks(rotation=90)
    plt.title(f"Test RMSE for {dataset_name}")
    
    plt.tight_layout()
    plt.show()

# Main workflow
def main():
    # Load the training datasets
    dataset1_train = pd.read_csv(r"C:\Users\sanay\Desktop\MACHINE LEARNING\DATASETS\EMBEDDINGS_GPT2_TRAIN.csv")
    dataset2_train = pd.read_csv(r"C:\Users\sanay\Desktop\MACHINE LEARNING\DATASETS\EMBEDDINGS_FASTTEXT_TRAIN.csv")
    
    # Load the test datasets (100 rows each)
    dataset1_test = pd.read_csv(r"C:\Users\sanay\Desktop\MACHINE LEARNING\DATASETS\GPT2_EMBEDDINGS_TEST.csv")
    dataset2_test = pd.read_csv(r"C:\Users\sanay\Desktop\MACHINE LEARNING\DATASETS\EMBEDDINGS_FASTTEXT_TEST.csv")
    
    # Preprocess the datasets
    dataset1_train = preprocess_data(dataset1_train)
    dataset2_train = preprocess_data(dataset2_train)
    dataset1_test = preprocess_data(dataset1_test)
    dataset2_test = preprocess_data(dataset2_test)
    
    # Specify columns and target for each dataset
    datasets = {
        "Dataset 1": {"train_data": dataset1_train, "test_data": dataset1_test, "features": dataset1_train.columns[:-1], "target": "Marks"},
        "Dataset 2": {"train_data": dataset2_train, "test_data": dataset2_test, "features": dataset2_train.columns[:-1], "target": "Marks"}
    }
    
    models = get_regression_models()
    all_results = []
    
    for dataset_name, details in datasets.items():
        train_data = details["train_data"]
        test_data = details["test_data"]
        X_train = train_data[details["features"]]
        y_train = train_data[details["target"]]
        X_test = test_data[details["features"]]
        y_test = test_data[details["target"]]
        
        # Train-test split for training data
        # Train the models
        results_df, _ = evaluate_models(X_train, X_test, y_train, y_test, models, dataset_name)
        all_results.append(results_df)
        
        # Generate predictions
        predictions = pd.DataFrame()
        predictions["Actual"] = y_test
        for name, model in models.items():
            model.fit(X_train, y_train)
            predictions[name] = model.predict(X_test)
        
        # Save predictions to an Excel file
        output_filename = f"{dataset_name}_predictions.xlsx"
        predictions.to_excel(output_filename, index=False)
        print(f"Predictions for {dataset_name} saved to {output_filename}")
        
        # Plot overall metrics
        plot_metrics(results_df, dataset_name)

# Run the program
if __name__ == "__main__":
    main()



Results for Dataset 1:
                              Model  Train RMSE  Test RMSE  Train R^2  \
0                Linear Regression    0.775579   3.330779   0.768410   
1                 Ridge Regression    0.925733   1.850794   0.670057   
2                 Lasso Regression    1.470109   1.877804   0.167916   
3            ElasticNet Regression    1.431566   1.891042   0.210976   
4  Support Vector Regression (SVR)    1.368824   1.808196   0.278622   
5         Decision Tree Regression    0.170666   2.251061   0.988786   
6         Random Forest Regression    0.507313   1.802461   0.900912   
7     Gradient Boosting Regression    0.934075   1.807505   0.664084   
8              AdaBoost Regression    1.235084   1.712880   0.412699   
9               XGBoost Regression    0.194484   1.766437   0.985438   

   Test R^2 (Scaled)  CV RMSE Mean  CV RMSE Std  CV R^2 Mean (Scaled)  \
0              0.001      1.726334     0.286310              0.625640   
1              0.001      1.203151  

C:\Users\sanay\AppData\Local\Temp\ipykernel_20428\1479671543.py:87: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=results_df, x="Model", y="Test R^2 (Scaled)", palette="viridis")
C:\Users\sanay\AppData\Local\Temp\ipykernel_20428\1479671543.py:93: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=results_df, x="Model", y="Test RMSE", palette="magma")



Results for Dataset 2:
                              Model  Train RMSE  Test RMSE  Train R^2  \
0                Linear Regression    1.171353  33.652998   0.471745   
1                 Ridge Regression    1.329819   2.303190   0.319148   
2                 Lasso Regression    1.611633   2.108964   0.000000   
3            ElasticNet Regression    1.611633   2.108964   0.000000   
4  Support Vector Regression (SVR)    1.348455   2.340428   0.299932   
5         Decision Tree Regression    0.170279   2.109502   0.988837   
6         Random Forest Regression    0.512033   2.354247   0.899060   
7     Gradient Boosting Regression    1.089047   3.014465   0.543373   
8              AdaBoost Regression    1.309214   2.241305   0.340084   
9               XGBoost Regression    0.241982   3.901263   0.977456   

   Test R^2 (Scaled)  CV RMSE Mean  CV RMSE Std  CV R^2 Mean (Scaled)  \
0              0.001      1.238267     0.052301              0.424880   
1              0.001      1.344101  

C:\Users\sanay\AppData\Local\Temp\ipykernel_20428\1479671543.py:87: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=results_df, x="Model", y="Test R^2 (Scaled)", palette="viridis")
C:\Users\sanay\AppData\Local\Temp\ipykernel_20428\1479671543.py:93: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `x` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=results_df, x="Model", y="Test RMSE", palette="magma")
